In [1]:
import os
import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split
from mne.filter import filter_data
from scipy.interpolate import griddata
import mne

# Define the paths
train_specInput_root_path = './SEED_input_data/train/feature'
train_tempInput_root_path = './SEED_input_data/train/temporal'
train_label_root_path = './SEED_input_data/train/label'

test_specInput_root_path = './SEED_input_data/test/feature'
test_tempInput_root_path = './SEED_input_data/test/temporal'
test_label_root_path = './SEED_input_data/test/label'


In [2]:

# Create directories if they do not exist
for path in [train_specInput_root_path, train_tempInput_root_path, train_label_root_path,
             test_specInput_root_path, test_tempInput_root_path, test_label_root_path]:
    os.makedirs(path, exist_ok=True)

# Define the input folder path
input_folder_path = '..\SST-EmotionNet\Preprocessed_EEG'

# Frequency bands
bands = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (14, 30),
    'gamma': (31, 50)
}

# Electrode positions
electrode_positions = {
    'FP1': (0, 3), 'FPZ': (0, 4), 'FP2': (0, 5),
    'AF3': (1, 3), 'AF4': (1, 5),
    'F7': (2, 0), 'F5': (2, 1), 'F3': (2, 2), 'F1': (2, 3), 'FZ': (2, 4), 'F2': (2, 5), 'F4': (2, 6), 'F6': (2, 7), 'F8': (2, 8),
    'FT7': (3, 0), 'FC5': (3, 1), 'FC3': (3, 2), 'FC1': (3, 3), 'FCZ': (3, 4), 'FC2': (3, 5), 'FC4': (3, 6), 'FC6': (3, 7), 'FT8': (3, 8),
    'T7': (4, 0), 'C5': (4, 1), 'C3': (4, 2), 'C1': (4, 3), 'CZ': (4, 4), 'C2': (4, 5), 'C4': (4, 6), 'C6': (4, 7), 'T8': (4, 8),
    'TP7': (5, 0), 'CP5': (5, 1), 'CP3': (5, 2), 'CP1': (5, 3), 'CPZ': (5, 4), 'CP2': (5, 5), 'CP4': (5, 6), 'CP6': (5, 7), 'TP8': (5, 8),
    'P7': (6, 0), 'P5': (6, 1), 'P3': (6, 2), 'P1': (6, 3), 'PZ': (6, 4), 'P2': (6, 5), 'P4': (6, 6), 'P6': (6, 7), 'P8': (6, 8),
    'PO7': (7, 0), 'PO5': (7, 1), 'PO3': (7, 2), 'POZ': (7, 3), 'PO4': (7, 4), 'PO6': (7, 5), 'PO8': (7, 6),
    'CB1': (8, 0), 'O1': (8, 1), 'OZ': (8, 2), 'O2': (8, 3), 'CB2': (8, 4)
}

# Example channel names corresponding to the EEG data indices
channel_names  = ['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2']

In [3]:

subject_files = {}

for file in os.listdir(input_folder_path):
    if file.endswith('.mat') and file != 'label.mat':
        subject_id = int(file.split('_')[0])
        if subject_id not in subject_files:
            subject_files[subject_id] = []
        subject_files[subject_id].append(file)

label_data = sio.loadmat(os.path.join(input_folder_path, 'label.mat'))['label'].flatten()

subjects_data = []

for subject_id, files in subject_files.items():
    files.sort()
    for session_id, file in enumerate(files):
        file_path = os.path.join(input_folder_path, file)
        data = sio.loadmat(file_path)
        eeg_keys = [key for key in data.keys() if 'eeg' in key]
        eeg_data = [data[key] for key in sorted(eeg_keys, key=lambda x: int(x.split('eeg')[-1]))]
        trial_indices = np.arange(15)
        train_indices, test_indices = train_test_split(trial_indices, test_size=0.1, random_state=40) # change form 42 to 40 for test
        print(len(train_indices), len(test_indices))
        subjects_data.append((eeg_data, label_data, subject_id, session_id, train_indices, test_indices))

13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2
13 2


In [4]:
import time
from sklearn.decomposition import PCA

def downsample_to_25_samples(data):
    n_channels, n_samples = data.shape
    segment_length = n_samples // 25
    downsampled_data = np.zeros((n_channels, 25))
    
    for i in range(25):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        downsampled_data[:, i] = np.mean(data[:, start_idx:end_idx], axis=1)
    
    return downsampled_data

def extract_de_features(data, sfreq, bands):
    de_features = []
    for band, (low_freq, high_freq) in bands.items():
        filtered_data = filter_data(data, sfreq, low_freq, high_freq, verbose=False)
        de_feature = np.log(np.var(filtered_data, axis=-1))
        de_features.append(de_feature)
    return np.array(de_features)

def cubic_spline_interpolation(array, target_resolution=(32, 32)):
    """
    Perform cubic spline interpolation to transform a 2D array of shape (9, 9, 1) to a higher resolution (32, 32, 1).
    
    Parameters:
    array (np.ndarray): Input array of shape (9, 9, 1).
    target_resolution (tuple): Target resolution (height, width).
    
    Returns:
    np.ndarray: Interpolated array of shape (target_resolution[0], target_resolution[1], 1).
    """
    h, w = target_resolution
    input_h, input_w = array.shape[:2]
    
    # Create the coordinate grid for the input array
    grid_x, grid_y = np.mgrid[0:input_h, 0:input_w]
    
    # Flatten the grid and the input array
    points = np.vstack((grid_x.ravel(), grid_y.ravel())).T
    values = array.ravel()
    
    # Create the coordinate grid for the target resolution
    grid_x_new, grid_y_new = np.mgrid[0:input_h-1:(h*1j), 0:input_w-1:(w*1j)]
    
    # Perform cubic spline interpolation
    interpolated_array = griddata(points, values, (grid_x_new, grid_y_new), method='cubic', fill_value=0)
    
    return interpolated_array[..., np.newaxis]

def transform_to_2d_map(features, electrode_positions, channel_names):
    h, w = 9, 9  # Grid size based on electrode positions
    grid_x, grid_y = np.mgrid[0:h, 0:w]
    positions = np.array([electrode_positions[ch] for ch in channel_names])
    map_2d = griddata(positions, features.reshape(-1, 1), (grid_x, grid_y), method='cubic', fill_value=0)
    return map_2d

def pad_arrays(array_list):
    max_length = max(array.shape[1] for array in array_list)
    padded_array = np.zeros((len(array_list), 62, max_length))
    for i, array in enumerate(array_list):
        padded_array[i, :, :array.shape[1]] = array
    return padded_array

def process_and_save_data(subject_data, subject_labels, subject_id, session_id, trial_indices, data_type='train'):
    spec_input_path = train_specInput_root_path if data_type == 'train' else test_specInput_root_path
    temp_input_path = train_tempInput_root_path if data_type == 'train' else test_tempInput_root_path
    label_path = train_label_root_path if data_type == 'train' else test_label_root_path

    selected_data = [subject_data[i] for i in trial_indices]
    selected_labels = subject_labels[trial_indices]
    
    print(f'Processing subject {subject_id}, session {session_id} with {len(selected_data)} trials')
    spatial_spectral_data = []
    spatial_temporal_data = []
    sfreq = 200  # Sampling frequency
    for trial in selected_data:
        print(f'Processing trial {trial.shape}')
        start_time = time.time()
        # TODO: normalize EEG Data
        info = mne.create_info(channel_names, sfreq=200, ch_types='eeg')
        raw = mne.io.RawArray(trial, info)
        raw.apply_function(lambda x: (x - np.mean(x) / np.std(x)))
        
        de_features = extract_de_features(trial, sfreq, bands)
        print(f'Extracted features {de_features.shape}')
        spectral_maps = np.array([cubic_spline_interpolation(transform_to_2d_map(de_feature, electrode_positions, channel_names)) for de_feature in de_features])
        spectral_maps = np.transpose(spectral_maps, (1, 2, 0, 3))
        print(f'Transformed spectral maps {len(spectral_maps)}')
        
        
        #TODO: do mean or PCA and maybe increase T samples
        # new_sfreq = 200 / (raw.get_data().shape[1] / 25)
        # raw.resample(sfreq=new_sfreq)

        # Downsample the raw data to 40 samples
        pca = PCA(n_components=40)
        print(raw.get_data().shape)
        downsampled_data = pca.fit_transform(raw.get_data())
        print(downsampled_data.shape)
        temporal_maps = np.array([cubic_spline_interpolation(transform_to_2d_map(downsampled_data[:, i], electrode_positions, channel_names)) for i in range(downsampled_data.shape[1])])
        print(f'Transformed temporal maps {len(temporal_maps)}')
        temporal_maps = np.transpose(temporal_maps, (1, 2, 0, 3))
        spatial_temporal_data.append(temporal_maps)
        spatial_spectral_data.append(spectral_maps)
        end_time = time.time()
        print(f'Trial processing took {end_time - start_time} seconds')

    spec_data = np.array(spatial_spectral_data)
    temp_data = np.array(spatial_temporal_data)
    label_data = selected_labels

    os.makedirs(os.path.join(spec_input_path, f'subject_{subject_id}'), exist_ok=True)
    os.makedirs(os.path.join(temp_input_path, f'subject_{subject_id}'), exist_ok=True)
    os.makedirs(os.path.join(label_path, f'subject_{subject_id}'), exist_ok=True)

    np.save(os.path.join(spec_input_path, f'subject_{subject_id}/section_{session_id}_data.npy'), spec_data)
    np.save(os.path.join(temp_input_path, f'subject_{subject_id}/section_{session_id}_data.npy'), temp_data)
    np.save(os.path.join(label_path, f'subject_{subject_id}/section_{session_id}_label.npy'), label_data)


In [7]:
(subject_data, subject_labels, subject_id, session_id, trial_indices, data_type) = subjects_data[0]
trial = subject_data[0]

info = mne.create_info(channel_names, sfreq=200, ch_types='eeg')
raw = mne.io.RawArray(trial, info)

de_features = extract_de_features(trial, 200, bands)
print(f'Extracted features {de_features.shape}')
spectral_maps = np.array([cubic_spline_interpolation(transform_to_2d_map(de_feature, electrode_positions, channel_names)) for de_feature in de_features])
spectral_maps = np.transpose(spectral_maps, (1, 2, 0, 3))
print(spectral_maps.shape)

# spec_data = np.array([spectral_maps, spectral_maps])
# spec_data.shape

Creating RawArray with float64 data, n_channels=62, n_times=47001
    Range : 0 ... 47000 =      0.000 ...   235.000 secs
Ready.
Extracted features (5, 62)
(32, 32, 5, 1)


In [8]:
info = mne.create_info(channel_names, sfreq=200, ch_types='eeg')

raw = mne.io.RawArray(subject_data[0], info)
raw.get_data().shape
pca = PCA(n_components=40)
print(raw.get_data().shape)
downsampled_data = pca.fit_transform(raw.get_data())
print(downsampled_data.shape)

# #plot both data and overlay
# import matplotlib.pyplot as plt
# plt.plot(downsampled_data.T)
# plt.show()
# #plot mne like matplotlib
# raw.plot(bgcolor='w', scalings='auto', title='Data from arrays', show=True, block=True)



Creating RawArray with float64 data, n_channels=62, n_times=47001
    Range : 0 ... 47000 =      0.000 ...   235.000 secs
Ready.
(62, 47001)
(62, 40)


In [5]:
for eeg_data, label_data, subject_id, session_id, train_indices, test_indices in subjects_data:
    process_and_save_data(eeg_data, label_data, subject_id, session_id, train_indices, data_type='train')
    process_and_save_data(eeg_data, label_data, subject_id, session_id, test_indices, data_type='test')

Processing subject 10, session 0 with 13 trials
Processing trial (62, 47601)
Creating RawArray with float64 data, n_channels=62, n_times=47601
    Range : 0 ... 47600 =      0.000 ...   238.000 secs
Ready.
Extracted features (5, 62)
Transformed spectral maps 32
(62, 47601)
(62, 40)
Transformed temporal maps 40
Trial processing took 1.7885165214538574 seconds
Processing trial (62, 41201)
Creating RawArray with float64 data, n_channels=62, n_times=41201
    Range : 0 ... 41200 =      0.000 ...   206.000 secs
Ready.
Extracted features (5, 62)
Transformed spectral maps 32
(62, 41201)
(62, 40)
Transformed temporal maps 40
Trial processing took 1.7025222778320312 seconds
Processing trial (62, 47601)
Creating RawArray with float64 data, n_channels=62, n_times=47601
    Range : 0 ... 47600 =      0.000 ...   238.000 secs
Ready.
Extracted features (5, 62)
Transformed spectral maps 32
(62, 47601)
(62, 40)
Transformed temporal maps 40
Trial processing took 1.4352030754089355 seconds
Processing tr